<img src="xxxx.jpg" width="200">
<h1><center>School</center></h1>

<h1><center>Envío de correos individualizados a partir de una base de datos en formato csv</center></h1>

## 1. Importar librería pandas, smtplib, ssl, email y time

In [20]:
## 1.1 Estas librerías permiten manipular la información y enviar los correos electrónicos con base en la base de datos en csv.
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import time

## 2. Importar base de datos para combinar datos

In [23]:
## 2.1 Colocar la base de datos en la misma carpeta que el presente archivo. Y actualizar el nombre de la base de datos.
informacion=pd.read_csv("XXXXXXXXXXX.csv", encoding= 'latin-1')

## 2.2 Los encabezados de la base de datos no deben llevar espacios.
informacion.tail(3)

,Nombre,Apellidos,Grupo,ID,Usuario,Profesor_de_ingles,correo1,correo2,correo3
320,ALDO IZAIM,VAZQUEZ LEON,"3 ""D""",3807380,ALDO-IZAIMVL,el mtro. Fabián Avendaño Corona,edcoyote16@yahoo.com,edcoyote16@yahoo.com,sin valor
321,JORGE RODRIGO,VIDRIOS GALARZA,"3 ""D""",4075587,JORGEVG,el mtro. Fabián Avendaño Corona,edcoyote16@yahoo.com,edcoyote16@yahoo.com,sin valor
322,DAVID LESEM,VILLEGAS PEREZ,"3 ""D""",3807412,DAVID-LESEMVP,el mtro. Fabián Avendaño Corona,edcoyote16@yahoo.com,edcoyote16@yahoo.com,sin valor


## 3. Crear función para que lea la base de datos, inserte la información y envíe los correos a los alumnos.

In [18]:
## Esta función lee, e inserta en el cuerpo de un mensaje las variables personalizadas por alumno.

def enviar_correos_individuales(fila):
    ## 3.1 Actualizar las columnas de acuerdo al orden de las columnas
    ## comenzando con la primera columna en 0. Estas variables se
    ## insertarán en el cuerpo del texto que se enviará.
    ## IMPORTANTE: La primera columna es la "0" (cero)
    nombre = fila[0]
    apellidos = fila[1]
    grupo = fila[2]
    profesor = fila[5]
    ID = fila[3]
    usuario = fila[4]
    correo1 = fila[6]
    correo2 = fila[7]
    correo3 = fila[8]
    
    ## 3.2 Agrega los correos que tenga el alumno registrados en la base de datos.
    correos=[]
    if correo1 != 'sin valor':
        correos.append(correo1)
    if correo2 != 'sin valor':
        correos.append(correo2)
    if correo3 != 'sin valor':
        correos.append(correo3)
    
    ## 3.3 Se marca copia a los correos de dirección o a quien corresponda. Nota: no es posible enviar con CC.
    correos.append('xxxxx@gmail.com')
    correos.append('xxxx@gmail.com')
    ## Es importante habilitar la opción de que aplicaciones de terceros controlen la cuenta de gmail.com desde
    ## dentro de la cuenta
    
    ## 3.4 Se establecen los correos de quién envía y quién recibe.
    sender_email = "xxx@gmail.com"
    password = "xxxxx"
    receiver_email = ", ".join(correos)

    ## 3.5 Se indica y se actualiza el tema del correo.
    message = MIMEMultipart("alternative")
    message["Subject"] = "Envío de xxxxx" ## IMPORTANTE: siempre actualizar el tema del correo.
    message["From"] = sender_email
    message["To"] = receiver_email
    
    ## 3.6 Se redacta el cuerpo del mensaje. Y se incluyen las variables de la base de datos.
    mensaje_en_html = """\
    <html>
      <body>

        <img style="display: block; margin-left: auto; margin-right: auto" 
        src="https://3.files.edl.io/e0c1/18/06/14/134826-2a01bd85-05cd-42ed-bbc6-a366e7e3bdbc.png" 
        alt="" width="172" height="172">
        <br>
        <center><p style="color:black;margin:15px 0;font:normal 24px arial, sans-serif;line-height:1.5em;padding:0 70px;"><b>
        Escuela Secundaria "10 de Diciembre"<br></b></p></center>
        <p style="color:black;margin:15px 0;font:normal 18px arial, sans-serif;line-height:1.5em;padding:0 70px;">
        <br>
        <br>
        Estimado tutor, madre y/o padre de familia de <b>{} {}, {}</b><br>
        <br>
           Esperando encontrarle con bien, reciba un cordial saludo de la dirección de secundaria "10 de Diciembre". 
           Asimismo por medio de la presente se hace de su conocimiento la siguiente información para el ingreso de {} a la
           plataforma UNOi.<br>
           <br>
           <br>
           <b style="color: blue;">&emsp;&emsp;El ID asignado para ingresar a la plataforma de sistema UNOi es {}</b>
           <br>
           <b style="color: blue;">&emsp;&emsp;El usuario de sistema UNOi es {} </b>
           <br>
           Solicitamos su valioso apoyo para que su hijo(a) pueda ingresar de forma regular.
           <br>
           <br>Como siempre, agradecemos su atención y el apoyo brindado a sus hijos.<br>
           <br>
           Atentamente,<br>
           <br>
           <br>
           <br>
           XXXXXXXXXXX<br>
           XXXXXXXXXXX
        </p>
      </body>
    </html>
    """.format(nombre,apellidos,grupo,nombre,ID,usuario)
    
    ## 3.7 Se incluye el mensaje para enviar en los parámetros de la librería.
    mensaje = MIMEText(mensaje_en_html, "html")
    message.attach(mensaje)

    # 3.8 Se crea una conexión segura entre el servidor y el cliente.
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string())
    ## 3.9 Actualizar el tiempo entre envío y envío de correo a 8 segundos.
        time.sleep( 6 )
    
    return grupo, nombre, apellidos, correos

## 4. Se envían los correos con base en la base de datos que se importó en el núm. 2

In [19]:
## 4.1 Se activa la función sobre la base de datos leyendo fila por fila.
informacion.apply(enviar_correos_individuales, axis=1)

KeyboardInterrupt: 

In [15]:
(6*325)/60

32.5